# Введение в методы анализа данных. Язык Python.

## Лекция 14: Асинхронное программирование. Asyncio

<br><br><br><br>
__Аксентьев Артем (akseart@ya.ru)__

__Ксемидов Борис (nstalker.anonim@yandex.ru)__
<br>

# План:
- "Синхронное" программирование
- Асинхронное программирование
    - Корутины
    - Задачи
- Пример кода
- Разбор первого ДЗ

In [19]:
%%writefile example_1.py
import time


def waiter():
    cook('Бургер', 5)
    cook('Салат', 7)
    cook('Картошка фри', 1)


def cook(order, time_to_prepare):
    print(f'Новый заказ: {order}')
    time.sleep(time_to_prepare)
    print(order, '- готово')


if __name__ == "__main__":
    waiter()

Overwriting example_1.py


# Что такое асинхронность?
1. Возможность не ожидать завершение какого-либо действия, а осуществлять другое действие
2. Возможность обрабатывать несколько запросов к боту/серверу

In [20]:
input()

'123'

In [ ]:
['server.com/file1', 'server.com/file2', 'server.com/file3']

## Блокировка выполнения программы
Приостановление вычислений(выполнения программы) в ожидание какого-либо действия(ввод-вывод). Чаще всего это:
- блокировка сетевого ввода-вывода,
- блокировка дискового ввода-вывода,
- блокировка пользовательского

## Неблокирующие функции:
Возможность продолжения выполнения программы при ожидании чего-либо.  То есть, при ожидании, например, ввода, программа может выполнять какие-либо другие действия.

## Асинхронная программа
Для выполнения задачи между различными программными блоками не требуется синхронизация.

Например, загрузка нескольких веб-страниц. После вызова загрузчика очередной страницы, нам не требуется ничего делать, до тех пор, пока она не будет загружена, поэтому можем перейти к какой-либо другой задачи. Время таких операций не определено.


__Асинхронный код работает в одном потоке и в одном процессе__

Перейдем к почти асинхронному коду:

In [21]:
%%writefile example_2.py
import asyncio


async def waiter():
    await cook('Бургер', 5)
    await cook('Салат', 7)
    await cook('Картошка фри', 1)


async def cook(order, time_to_prepare):
    print(f'Новый заказ: {order}')
    await asyncio.sleep(time_to_prepare)
    print(order, '- готово')


if __name__ == '__main__':
    asyncio.run(waiter())

Overwriting example_2.py


__Корутины (сопрограммы)__ -- функции, объявленные с ключевым словом async. То есть функция, которая содержит в себе задачи, которые могут выполняться в асинхронном режиме

__Задачи (tasks)__ -- некий спосооб для запуска нескольких корутин.

In [22]:
%%writefile example_3.py
import asyncio


async def waiter():
    task1 = asyncio.create_task(cook('Бургер', 5))
    task2 = asyncio.create_task(cook('Салат', 7))
    task3 = asyncio.create_task(cook('Картошка фри', 1))

    await task1
    await task2
    await task3


async def cook(order, time_to_prepare):
    print(f'Новый заказ: {order}')
    raise KeyError
    await asyncio.sleep(time_to_prepare)
    print(order, '- готово')


if __name__ == '__main__':
    try:
        asyncio.run(waiter())
    except:
        ...

Overwriting example_3.py


<img alt="1" src="/Users/artem/Documents/Chillers_Lections/Python_MSU/course-python/lectures/Lection_14_Asyncio/Untitled Diagram.drawio.png"/>

In [25]:
def gen():
    for i in range(10):
        yield i

a = gen()
print(next(a))
print("Hello")

0
Hello


In [26]:
print(next(a))

1


# Плюсы:
- Один поток и один процесс
- Меньше накладных расходов
- Не зависит от ОС
# Минусы:
- Сложность отладки(не всегда понятно что происходит)
- Трудно понять какая функция сейчас исполняется
- Более сложный код
- Следует помнить про блокирующие библиотеки(requests, time.sleep)

# Ещё один пример:

In [27]:
%%writefile example_4.py
import socket


def main():
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    server_socket.bind(('localhost', 50003))
    server_socket.listen()

    while True:
        client_socket, addr = server_socket.accept()
        while True:
            request = client_socket.recv(4096)
            if request:
                response = "Hello \n".encode()
                client_socket.send(response)
            else:
                break


if __name__ == "__main__":
    main()

Overwriting example_4.py


In [28]:
%%writefile example_5.py
import asyncio


async def handle_echo(reader, writer):
    while True:
        data = await reader.read(100)
        message = data.decode()
        addr = writer.get_extra_info('peername')
        if message.strip() == "exit":
            print(f"Exit from {addr!r}")
            writer.close()
            break


        print(f"Received {message!r} from {addr!r}")

        print(f"Send: {message!r}")
        writer.write(data)
        await writer.drain()


async def main():
    server = await asyncio.start_server(
        handle_echo, '127.0.0.1', 50003)

    addr = server.sockets[0].getsockname()
    print(f'Serving on {addr}')

    async with server:
        await server.serve_forever()


if __name__ == "__main__":
    asyncio.run(main())

Overwriting example_5.py


# Вопросы для самостоятельного изучения:
- asyncio
- aiohttp
- aiogram

# Рекомендуемая литература:
- [PEP 492 – Coroutines with async and await syntax](https://peps.python.org/pep-0492/)
- [Асинхронность в Python. Олег Молчанов](https://youtube.com/playlist?list=PLlWXhlUMyooawilqK4lPXRvxtbYiw34S8)
- [Асинхронщина в Python Александр Полищук](https://www.youtube.com/watch?v=lIkA0TDX8tE)
- [docs.python.org](https://docs.python.org/3/library/asyncio-task.html)

# Вопросы к зачету